# Clustering provinces_prepared_New

### Notebook automatically generated from your model

Model KMeans (k=3), trained on 2019-06-26 16:11:05.

#### Generated on 2019-06-26 14:23:56.653193

Clustering
This notebook will reproduce the steps for clustering the dataset provinces_prepared_New.

#### Warning

The goal of this notebook is to provide an easily readable and explainable code that reproduces the main steps
of training the model. It is not complete: some of the preprocessing done by the DSS visual machine learning is not
replicated in this notebook. This notebook will not give the same results and model performance as the DSS visual machine
learning model.

Let's start with importing the required libs :

In [ ]:
import sys
import dataiku
import numpy as np
import pandas as pd
import sklearn as sk
import dataiku.core.pandasutils as pdu
from dataiku.doctor.preprocessing import PCA
from collections import defaultdict, Counter

And tune pandas display options:

In [ ]:
pd.set_option('display.width', 3000)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

#### Importing base data

The first step is to get our machine learning dataset:

In [ ]:
# We apply the preparation that you defined. You should not modify this.
preparation_steps = []
preparation_output_schema = {u'userModified': False, u'columns': [{u'type': u'string', u'name': u'province'}, {u'type': u'double', u'name': u'pmale'}, {u'type': u'bigint', u'name': u'population'}, {u'type': u'double', u'name': u'pforeign'}, {u'type': u'double', u'name': u'pobgrowth'}, {u'type': u'bigint', u'name': u'ind_wholesale'}, {u'type': u'bigint', u'name': u'ind_retail'}, {u'type': u'bigint', u'name': u'ind_rest'}, {u'type': u'bigint', u'name': u'ind_turis'}, {u'type': u'double', u'name': u'indtur_stan'}, {u'type': u'bigint', u'name': u'ind_actindex'}, {u'type': u'double', u'name': u'unemprate'}, {u'type': u'double', u'name': u'adsl'}, {u'type': u'double', u'name': u'p_adsl'}, {u'type': u'double', u'name': u'pcars'}, {u'type': u'double', u'name': u'pcars_stan'}, {u'type': u'double', u'name': u'penergy'}, {u'type': u'double', u'name': u'pmining'}, {u'type': u'double', u'name': u'pmetal'}, {u'type': u'double', u'name': u'pmanufac'}, {u'type': u'double', u'name': u'pbuilding'}, {u'type': u'double', u'name': u'pagric'}, {u'type': u'double', u'name': u'ptextile'}, {u'type': u'double', u'name': u'ppharmac'}, {u'type': u'double', u'name': u'pdurab'}, {u'type': u'double', u'name': u'pinterind'}, {u'type': u'double', u'name': u'pinindother'}, {u'type': u'double', u'name': u'pothwhole'}, {u'type': u'double', u'name': u'pretail_food'}, {u'type': u'double', u'name': u'pretail_nonf'}, {u'type': u'double', u'name': u'pretail_other'}, {u'type': u'double', u'name': u'p_banks_GDP'}]}

ml_dataset_handle = dataiku.Dataset('provinces_prepared_New')
ml_dataset_handle.set_preparation_steps(preparation_steps, preparation_output_schema)
%time ml_dataset = ml_dataset_handle.get_dataframe(limit = 100000)

print ('Base data has %i rows and %i columns' % (ml_dataset.shape[0], ml_dataset.shape[1]))
# Five first records",
ml_dataset.head(5)

#### Initial data management

The preprocessing aims at making the dataset compatible with modeling.
At the end of this step, we will have a matrix of float numbers, with no missing values.
We'll use the features and the preprocessing steps defined in Models.

Let's only keep selected features

In [ ]:
ml_dataset = ml_dataset[[u'indtur_stan', u'p_banks_GDP', u'pmining', u'pretail_other', u'pinterind', u'penergy']]

Let's first coerce categorical columns into unicode, numerical features into floats.

In [ ]:
# astype('unicode') does not work as expected

def coerce_to_unicode(x):
    if sys.version_info < (3, 0):
        if isinstance(x, str):
            return unicode(x,'utf-8')
        else:
            return unicode(x)
    else:
        return str(x)


categorical_features = []
numerical_features = [u'indtur_stan', u'p_banks_GDP', u'pmining', u'pretail_other', u'pinterind', u'penergy']
text_features = []
from dataiku.doctor.utils import datetime_to_epoch
for feature in categorical_features:
    ml_dataset[feature] = ml_dataset[feature].apply(coerce_to_unicode)
for feature in text_features:
    ml_dataset[feature] = ml_dataset[feature].apply(coerce_to_unicode)
for feature in numerical_features:
    if ml_dataset[feature].dtype == np.dtype('M8[ns]'):
        ml_dataset[feature] = datetime_to_epoch(ml_dataset[feature])
    else:
        ml_dataset[feature] = ml_dataset[feature].astype('double')

Let's copy our dataset to keep it for eventual profiling at the end.

In [ ]:
# train dataset will be the one on which we will apply ml technics
train = ml_dataset.copy()

#### Features preprocessing

The first thing to do at the features level is to handle the missing values.
Let's reuse the settings defined in the model

In [ ]:
drop_rows_when_missing = []
impute_when_missing = [{'impute_with': u'MEAN', 'feature': u'indtur_stan'}, {'impute_with': u'MEAN', 'feature': u'p_banks_GDP'}, {'impute_with': u'MEAN', 'feature': u'pmining'}, {'impute_with': u'MEAN', 'feature': u'pretail_other'}, {'impute_with': u'MEAN', 'feature': u'pinterind'}, {'impute_with': u'MEAN', 'feature': u'penergy'}]

# Features for which we drop rows with missing values"
for feature in drop_rows_when_missing:
    train = train[train[feature].notnull()]
    
    print ('Dropped missing records in %s' % feature)

# Features for which we impute missing values"
for feature in impute_when_missing:
    if feature['impute_with'] == 'MEAN':
        v = train[feature['feature']].mean()
    elif feature['impute_with'] == 'MEDIAN':
        v = train[feature['feature']].median()
    elif feature['impute_with'] == 'CREATE_CATEGORY':
        v = 'NULL_CATEGORY'
    elif feature['impute_with'] == 'MODE':
        v = train[feature['feature']].value_counts().index[0]
    elif feature['impute_with'] == 'CONSTANT':
        v = feature['value']
    train[feature['feature']] = train[feature['feature']].fillna(v)
    
    print ('Imputed missing values in feature %s with value %s' % (feature['feature'], coerce_to_unicode(v)))

We can now handle the categorical features (still using the settings defined in Models):

In [ ]:
# Rescaling is not required

Removing outliers

In [ ]:
# Remove outliers from train set
from dataiku.doctor.preprocessing.dataframe_preprocessing import detect_outliers

outliers = detect_outliers(train, 0.9, 0, 0.01)
train = train[~outliers]

print ("%s outliers found" % (outliers.sum()))

#### Modeling

In [ ]:
from sklearn.cluster import KMeans
clustering_model = KMeans(n_clusters=3)

We can finally cluster our dataset!

In [ ]:
%time clusters = clustering_model.fit_predict(train)

Build up our result dataset

#### Results

Inertia

In [ ]:
print (clustering_model.inertia_)

Silhouette

In [ ]:
from sklearn.metrics import silhouette_score
silhouette = silhouette_score(train.values, clusters, metric='euclidean', sample_size=2000)
print ("Silhouette score :", silhouette)

Join our original dataset with the cluster labels we found.

In [ ]:
final = train.join(pd.Series(clusters, index=train.index, name='cluster'))
final['cluster'] = final['cluster'].map(lambda cluster_id: 'cluster' + str(cluster_id))

Compute the cluster sizes

In [ ]:
size = pd.DataFrame({'size': final['cluster'].value_counts()})
size.head()

Draw a nice scatter plot

In [ ]:
axis_x = train.columns[0]   # change me
axis_y = train.columns[1]  # change me

from ggplot import ggplot, aes, geom_point
print(ggplot(aes(axis_x, axis_y, colour='cluster'), final) + geom_point())

That's it. It's now up to you to tune your preprocessing, your algo, and your analysis !
